In [1]:
from src.jobs.utils.utility import create_spark_session
spark = create_spark_session("dev_app")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/12 00:43:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Listar tabelas
spark.sql("SHOW TABLES IN nessie").show()


+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [3]:
spark.sql("""CREATE NAMESPACE IF NOT EXISTS nessie.bronze""")

spark.sql("""CREATE NAMESPACE IF NOT EXISTS nessie.silver""")

spark.sql("""CREATE NAMESPACE IF NOT EXISTS nessie.gold""")

DataFrame[]

In [4]:
# Bronze Tables

spark.sql(open("settings/create_bronze_vocation.sql").read())
spark.sql(open("settings/create_bronze_extra.sql").read())
spark.sql(open("settings/create_bronze_skills.sql").read())

# Silver Tables
spark.sql(open("settings/create_silver_vocation.sql").read())
spark.sql(open("settings/create_silver_extra.sql").read())
spark.sql(open("settings/create_silver_skills.sql").read())



SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


DataFrame[]

In [5]:
# Listar tabelas
spark.sql("SHOW TABLES IN nessie").show()


+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|   bronze|    extra|      false|
|   bronze|   skills|      false|
|   bronze| vocation|      false|
|   silver|    extra|      false|
|   silver|   skills|      false|
|   silver| vocation|      false|
+---------+---------+-----------+



In [6]:
spark.stop()

In [1]:
from src.jobs.utils.utility import create_spark_session
spark = create_spark_session("dev_app")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/20 22:58:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark.sql("DROP TABLE IF EXISTS nessie.bronze.extra")
spark.sql("DROP TABLE IF EXISTS nessie.silver.extra")

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


DataFrame[]

In [3]:
spark.sql("SHOW TABLES IN nessie").show()


+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|   bronze|   skills|      false|
|   bronze| vocation|      false|
|   silver|   skills|      false|
|   silver| vocation|      false|
+---------+---------+-----------+



In [4]:
spark.stop()

In [2]:
spark.sql(""" CREATE OR REPLACE TABLE nessie.gold.vocation_rank_global
USING iceberg
AS
SELECT
      ROW_NUMBER() OVER (ORDER BY level DESC) AS rank
    , name
    , world
    , vocation
    , level
    , experience
    , world_type
    , current_timestamp() AS updated_at
FROM nessie.silver.vocation
WHERE is_current = true;""")


SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
26/01/20 22:59:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/20 22:59:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/20 22:59:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/20 23:00:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/20 23:00:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can caus

DataFrame[]

In [3]:
spark.sql("SHOW TABLES IN nessie").show()

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|   bronze|              skills|      false|
|   bronze|            vocation|      false|
|     gold|vocation_rank_global|      false|
|   silver|              skills|      false|
|   silver|            vocation|      false|
+---------+--------------------+-----------+



In [5]:
spark.sql(""" SELECT * FROM nessie.gold.vocation_rank_global """).show()

+----+-----------------+--------+---------------+-----+------------+------------+--------------------+
|rank|             name|   world|       vocation|level|  experience|  world_type|          updated_at|
+----+-----------------+--------+---------------+-----+------------+------------+--------------------+
|   1|        Dejairzin|  inabra|    elder druid| 2997|447914003765|    Open PvP|2026-01-20 22:59:...|
|   2|           Bobeek|    bona|    elder druid| 2958|430572283021|Optional PvP|2026-01-20 22:59:...|
|   3|           Goraca|    bona|master sorcerer| 2955|429183508697|Optional PvP|2026-01-20 22:59:...|
|   4|          Veyllor|gentebra|  royal paladin| 2909|409683078307|Optional PvP|2026-01-20 22:59:...|
|   5|   Warlord Teuzin| belobra|master sorcerer| 2877|396292619074|Optional PvP|2026-01-20 22:59:...|
|   6|            Vepeh| celebra|  royal paladin| 2861|389654187727|Optional PvP|2026-01-20 22:59:...|
|   7|       Mago Morto| celebra|master sorcerer| 2798|364591370274|Optio

In [7]:
spark.stop()

In [2]:
from src.jobs.utils.utility import create_spark_session
spark = create_spark_session("dev")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/26 18:19:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.sql("SHOW TABLES IN nessie").show()

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|   bronze|              skills|      false|
|   bronze|            vocation|      false|
|     gold|experience_rank_g...|      false|
|     gold|  skills_rank_global|      false|
|     gold|vocation_rank_global|      false|
|     gold|       world_summary|      false|
|   silver|              skills|      false|
|   silver|            vocation|      false|
+---------+--------------------+-----------+



In [4]:
tables_gold = [
    "nessie.gold.experience_rank_global",
    "nessie.gold.skills_rank_global",
    "nessie.gold.vocation_rank_global",
    "nessie.gold.world_summary"
]

for table in tables_gold:
    spark.sql(f"DROP TABLE IF EXISTS {table}")

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


In [5]:
spark.sql("SHOW TABLES IN nessie").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|   bronze|   skills|      false|
|   bronze| vocation|      false|
|   silver|   skills|      false|
|   silver| vocation|      false|
+---------+---------+-----------+



In [6]:
spark.stop()

In [7]:
from src.jobs.utils.utility import create_spark_session
spark = create_spark_session("dev")

spark.sql("SHOW TABLES IN nessie").show()

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|   bronze|              skills|      false|
|   bronze|            vocation|      false|
|     gold|experience_rank_g...|      false|
|     gold|  skills_rank_global|      false|
|     gold|       world_summary|      false|
|   silver|              skills|      false|
|   silver|            vocation|      false|
+---------+--------------------+-----------+



In [8]:
spark.stop()

In [9]:
from src.jobs.utils.utility import create_spark_session
spark = create_spark_session("dev")

In [11]:
!pip install pandas
import pandas as pd

     |████████████████████████████████| 12.4 MB 9.6 MB/s eta 0:00:01    |████████████████████████▎       | 9.4 MB 9.6 MB/s eta 0:00:01
     |████████████████████████████████| 348 kB 11.2 MB/s eta 0:00:01
     |████████████████████████████████| 17.3 MB 7.8 MB/s eta 0:00:01    |█████████████▌                  | 7.3 MB 9.1 MB/s eta 0:00:02


# EXPORTANDO AS TABELAS PARA CSV

In [12]:
# BRONZE
(
    spark.sql("""
        SELECT *
        FROM nessie.bronze.vocation
    """).toPandas()).to_csv("bronze_vocation.csv")
(
    spark.sql("""
        SELECT *
        FROM nessie.bronze.skills
    """).toPandas()).to_csv("bronze_skills.csv")

# SILVER

(
    spark.sql("""
        SELECT *
        FROM nessie.silver.skills
    """).toPandas()).to_csv("silver_skills.csv")

(
    spark.sql("""
        SELECT *
        FROM nessie.silver.vocation
    """).toPandas()).to_csv("silver_vocation.csv")

# GOLD

(
    spark.sql("""
        SELECT *
        FROM nessie.gold.skills_rank_global
    """).toPandas()).to_csv("gold_skills_rank_global.csv")

(
    spark.sql("""
        SELECT *
        FROM nessie.gold.experience_rank_global
    """).toPandas()).to_csv("gold_experience_rank_global.csv")

(
    spark.sql("""
        SELECT *
        FROM nessie.gold.world_summary
    """).toPandas()).to_csv("gold_world_summary.csv")


26/01/26 19:28:45 WARN S3FileIO: Unclosed S3FileIO instance created by:
	org.apache.iceberg.aws.s3.S3FileIO.initialize(S3FileIO.java:359)
	org.apache.iceberg.CatalogUtil.loadFileIO(CatalogUtil.java:351)
	org.apache.iceberg.nessie.NessieCatalog.initialize(NessieCatalog.java:132)
	org.apache.iceberg.CatalogUtil.loadCatalog(CatalogUtil.java:256)
	org.apache.iceberg.CatalogUtil.buildIcebergCatalog(CatalogUtil.java:310)
	org.apache.iceberg.spark.SparkCatalog.buildIcebergCatalog(SparkCatalog.java:154)
	org.apache.iceberg.spark.SparkCatalog.initialize(SparkCatalog.java:753)
	org.apache.spark.sql.connector.catalog.Catalogs$.load(Catalogs.scala:65)
	org.apache.spark.sql.connector.catalog.CatalogManager.$anonfun$catalog$1(CatalogManager.scala:54)
	scala.collection.mutable.HashMap.getOrElseUpdate(HashMap.scala:86)
	org.apache.spark.sql.connector.catalog.CatalogManager.catalog(CatalogManager.scala:54)
	org.apache.spark.sql.connector.catalog.LookupCatalog$CatalogAndNamespace$.unapply(LookupCatalog.

In [13]:
spark.stop()